# <center>Successive Convexification of Non-Convex Optimal Control Problems with State Constraints<center>

## General optimal control problem

"Optimal control theory is a branch of mathematical optimization that deals with finding a control for a dynamical system over a period of time such that an objective function is optimized."

\begin{aligned}
& \min J\left(x, u\right), \\
& \text { subject to } \\
& x_{i+1}=f\left(x_i, u_i\right) \quad i=1,2, T-1, \\
& u_i \in U_i \subseteq \mathbb{R}^m \quad i=1,2, T-1 \text {, } \\
& x_i \in X_i \subseteq \mathbb{R}^n \quad i=1,2, T \text {. } \\
&
\end{aligned}

## An aerospace problem
\begin{array}{ll}
\min & \sum_{i=1}^N\left\|u_i\right\| \\
\text { s.t. } & x_{i+1}=A x_i+B\left(u_i+g\right), i=1, \ldots, N-1 \\
& \left\|u_i\right\|_2 \leq u_{\max }, i=1, \ldots, N, \\
& \left\|v_i\right\|_2 \leq V_{\max }, i=1, \ldots, N, \\
& \hat{n}^T u_i \geq\left\|u_i\right\|_2 \cos \left(\theta_{c o n e}\right), i=1, \ldots, N, \\
& \left\|H x_i-p_{c, j}\right\|_2 \geq r_j, i=1, \ldots, N, j=1, \ldots, n_{c y l}, \\
& x_0=x\left(t_0\right), x_N=x\left(t_f\right) .
\end{array}

\begin{array}{lc|lc}
\hline \hline \text { Par. } & \text { Value } & \text { Par. } & \text { Value } \\
\hline N & 25 & \epsilon & 1 \times 10^{-6} \\
V_{\max } & 2 \mathrm{~m} / \mathrm{s} & u_{\max } & 13.33 \mathrm{~m} / \mathrm{s}^2 \\
g & {[0,0,-9.81]^T \mathrm{~m} / \mathrm{s}^2} & \theta_{\text {cone }} & 30 \mathrm{deg} \\
p_0 & {[-8,-1,0]^T \mathrm{~m}} & p_f & {[8,1,0.5]^T \mathrm{~m}} \\
v_0 & {[0,0,0]^T \mathrm{~m} / \mathrm{s}} & v_f & {[0,0,0]^T \mathrm{~m} / \mathrm{s}} \\
p_{c, 1} & {[-1,0]^T \mathrm{~m}} & p_{c, 2} & {[4,-1]^T \mathrm{~m}} \\
r_1 & 3 \mathrm{~m} & r_2 & 1.5 \mathrm{~m} \\
\lambda & 0 & \hat{n} & {[0,0,1]^T} \\
t_f & 15 \mathrm{~s} & & \\
\hline \hline
\end{array}

## Import packages

In [ ]:
using Plots
using PlotlyJS
using JuMP
using Ipopt
using HiGHS
using AmplNLWriter
using Couenne_jll
plotlyjs()

## Data Structure

In [ ]:
struct Data
    R::Vector{Float64}
    Px::Vector{Float64}
    Py::Vector{Float64}
    Xo::Vector{Float64}
    Xf::Vector{Float64}
    Vo::Vector{Float64}
    Vf::Vector{Float64}
    N::Int
    Vmax::Float64
    Umax::Float64
    g::Vector{Float64}
    T::Float64
    lambda::Float64
    theta::Float64
    n::Vector{Float64}
end

In [ ]:
data = Data([3.0, 1.5],#, 1.5],
            [-1, 4],#, 4],
            [0, -1],#, 3],
            [-8.0, -1.0, 0.0],
            [8.0, 1.0, 0.5],
            [0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0],
            25,
            2,
            13.33,
            [0.0, 0.0, 9.81],
            15.0,
            1000.0,
            pi/6,
            [0.0, 0.0, 1.0]
            );

 ## Create Plot

In [ ]:
function create_plot(data::Data)

    plt = Plots.plot()
    for i in 1:length(data.R)
        m, n = 200, 150
        u = range(0, 2pi, length=n)
        v = range(-5, 5, length=m)
    
        us = ones(m)*u'
        vs = v*ones(n)'
        X = data.R[i]*cos.(us) .+ data.Px[i]
        Y = data.R[i]*sin.(us) .+ data.Py[i]
        Z = vs
        Plots.surface!(plt, X, Y, Z, size=(600,600), cbar=:none, legend=false)
    end
    Plots.scatter3d!(plt, [data.Xo[1]], [data.Xo[2]], [data.Xo[3]])
    Plots.scatter3d!(plt, [data.Xf[1]], [data.Xf[2]], [data.Xf[3]])
    return plt
end
plt = create_plot(data)

## Try to solve with interior points

In [ ]:
function try_solve_ipopt(data::Data)
    model = Model(Ipopt.Optimizer);

    @variable(model, x[1:3, 1:data.N])
    @variable(model, v[1:3, 1:data.N])
    @variable(model, u[1:3, 1:data.N-1])

    #Y
    @constraint(model, x[:,1] .== data.Xo)
    @constraint(model, x[:,end] .== data.Xf)
    @constraint(model, v[:,1] .== data.Vo)
    @constraint(model, v[:,end] .== data.Vf)
    @constraint(model, [i in 1:data.N], sum(v[j,i]^2 for j in 1:3) <= data.Vmax^2)
    @constraint(model, [i in 1:data.N-1], sum(u[j,i]^2 for j in 1:3) <= data.Umax^2)
    @constraint(model, [i in 1:data.N-1], sum(u[:,i].*data.n)^2 >= sum(u[j,i]^2 for j in 1:3) * cos(data.theta)^2)

    #q
    @constraint(model,[i in 1:data.N-1], v[:,i]*data.T/data.N - x[:,i+1] + x[:,i] .== 0)
    @constraint(model,[i in 1:data.N-1], u[:,i]*data.T/data.N - data.g*data.T/data.N - v[:,i+1] + v[:,i] .== 0)
    for k in 1:length(data.R)
        @constraint(model, [i in 1:data.N], (x[1,i]-data.Px[k])^2 + (x[2,i]-data.Py[k])^2 - data.R[k]^2 >= 0)
    end
    @NLobjective(model, Min, sum(sqrt(sum(u[j,k]^2 for j in 1:3)) for k in 1:data.N-1)^2)

    optimize!(model);
    return 
end
try_solve_ipopt(data)

## Reframing the problem

1)
\begin{aligned}
& \min J\left(x, u\right), \\
& \text { subject to } \\
& x_{i+1}-x_{i}=f\left(x_i, u_i\right) \quad i=1,2, T-1, \\
& h\left(x_i\right) \geq 0 \quad i=1,2, T \\
& u_i \in U_i \subseteq \mathbb{R}^m \quad i=1,2, T-1 \text {, } \\
& x_i \in X_i \subseteq \mathbb{R}^n \quad i=1,2, T \text {. } \\
&
\end{aligned}

<ol>
  <p>Let $y=$ $\left(x_1^T, \ldots, x_T^T, u_1^T, \ldots, u_{T-1}^T,\right)^T \in \mathbb{R}^N$, where $N=m(T-1)+$ $n T$.</p>
  <p>Let $Y$ be the Cartesian product of all $X_i$ and $U_i$, then $y \in Y$. $Y$ is a convex and compact set because $X_i$ and $U_i$ are.</p>
  <p>Let $g_i\left(x_i, u_i\right)=f\left(x_i, u_i\right)-x_{i+1}+x_i$, and $g(y)=\left(g_1^T, g_2^T, \ldots, g_{T-1}^T\right)^T \in \mathbb{R}^{n(T-1)}$</p>
</ol>

2) 

\begin{aligned}
& \min J\left(y\right), \\
& \text { subject to } \\
& g(y)=0 \\
& h\left(y\right) \geq 0 \\
& y \in Y \\
\end{aligned}

3)

\begin{aligned}
& \min P(y)=J(y)+\lambda\|g(y)\|_1, \\
& \text { subject to } \\
& g(y) \geq 0 \\
& h\left(y\right) \geq 0 \\
& y \in Y \\
\end{aligned}

Theorem. (Exactness). Let $P(y)=J(y)+\lambda\|g(y)\|_1$ be the penalty function, and $\bar{y}$ be a stationary point of
$$
\min _y\{P(y) \mid y \in Y, g(y) \geq 0, h(y) \geq 0\}
$$
with Lagrangian multiplier $\bar{\mu}$ for equality constraints. Then, if the penalty weight $\lambda$ satisfies $\lambda \geq\|\bar{\mu}\|_{\infty}$, and if $\bar{y}$ is feasible for (2), then $\bar{y}$ is a critical point of (2).


4)

\begin{aligned}
& \min P(y)=J(y)+\lambda\|g(y)\|_1, \\
& \text { subject to } \\
& q\left(y\right) \geq 0 \\
& y \in Y \\
\end{aligned}

## Hypothesis

<ol>
  <p>Hypothesis 1. $f_j\left(x_i, u_i\right)$ is a convex function over $x_i$ and $u_i, \forall j=1,2, \ldots n$.</p>
  <p>Hypothesis 2. $h_j\left(x_i\right)$ is a convex function over $x_i, \forall j=$ $1,2, \ldots s$</p>
</ol>

## Project and Linearize

Theorem . (Uniqueness of projection). For any $z \in$ $F$, there exists a unique point
$$
\bar{z}_j=\underset{y}{\operatorname{argmin}}\left\{\|z-y\|_2 \mid q_j(y) \leq 0\right\},
$$
called the projection of z onto $q_j (y) ≤ 0$.

For a fixed $z \in F$, let $l_j(y, z)$ be the linear approximation of $q_j(y)$ :
$$
l_j(y, z)=\nabla_y q_j\left(\bar{z}_j\right)\left(y-\bar{z}_j\right),
$$
and let $l(y, z)=\left(l_1^T, l_2^T, \ldots, l_M^T\right)^T \in \mathbb{R}^M$. For each $z \in F$ denote
$$
F_z=\{y \mid y \in Y, l(y, z) \geq 0\}
$$

## Algorithm

\begin{equation}
\begin{aligned}
& \hline \text { Algorithm } 1 \text { Successive Convexification Algorithm } \\
& \hline \text { 1: procedure } \operatorname{SCvx}\left(z^{(0)}, \lambda\right) \\
& \text { 2: } \quad \text { input Initial point } z^{(0)} \in F \text {. Penalty weight } \lambda \geq 0 \text {. } \\
& \text { 3: } \quad \text { while not converged do } \\
& \text { 4: } \quad \quad \text { step 1 At each succession } k \text {, we have the } \\
& \quad \text { current point } z^{(k)} \text {. For each constraint } q_j(y) \text {, solve the } \\
& \quad \text { problem }
\bar{z}_j=\underset{y}{\operatorname{argmin}}\left\{\|z-y\|_2 \mid q_j(y) \leq 0\right\} \text {. } \\
& \text { 5: } \quad \quad \text { step } 2 \text { Construct the convex feasible region } \\
& \quad F_{z^{(k)}}  = \{y \mid y \in Y, l(y, z) \geq 0\} \text {. } \\
& \text { 6: } \quad \quad \text { step } 3 \text { Solve the convex subproblem  } \\
& z^{(k+1)}=\underset{y}{\operatorname{argmin}}\left\{P(y) \mid y \in F_{z^{(k)}}\right\}, \text { and go to the next } \\
& \quad \text { succession. } \\
& \text { 7: } \quad \text { end while } \\
& \text { 8: } \quad \text { return } z^{(k+1)}. \\
& \text { 9: end procedure }
\end{aligned}
\end{equation}


$$$$

## Convergence Analysis


<ol>
  <p>Lipschitz continuity of $l(y, z)$.</p>
  <p>Banach fixed-point theorem.</p>
  <p>See paper for more details.</p>
</ol>

## Get feasible solution

In [ ]:
function get_feasible_solution!(data::Data, plt::Plots.Plot)
    model = Model(() -> AmplNLWriter.Optimizer(Couenne_jll.amplexe));

    @variable(model, x[1:3, 1:data.N])
    @variable(model, v[1:3, 1:data.N])
    @variable(model, u[1:3, 1:data.N-1])

    #Y
    @constraint(model, x[:,1] .== data.Xo)
    @constraint(model, x[:,end] .== data.Xf)
    @constraint(model, v[:,1] .== data.Vo)
    @constraint(model, v[:,end] .== data.Vf)
    @constraint(model, [i in 1:data.N], sum(v[j,i]^2 for j in 1:3) <= data.Vmax^2)
    @constraint(model, [i in 1:data.N-1], sum(u[j,i]^2 for j in 1:3) <= data.Umax^2)
    @constraint(model, [i in 1:data.N-1], sum(u[:,i].*data.n)^2 >= sum(u[j,i]^2 for j in 1:3) * cos(data.theta)^2)

    #q
    @constraint(model,[i in 1:data.N-1], v[:,i]*data.T/data.N - x[:,i+1] + x[:,i] .== 0)
    @constraint(model,[i in 1:data.N-1], u[:,i]*data.T/data.N - data.g*data.T/data.N - v[:,i+1] + v[:,i] .== 0)
    for k in 1:length(data.R)
        @constraint(model, [i in 1:data.N], (x[1,i]-data.Px[k])^2 + (x[2,i]-data.Py[k])^2 - data.R[k]^2 >= 0)
    end

    optimize!(model);
    @show sum(sqrt(sum(value(u[j,k])^2 for j in 1:3)) for k in 1:data.N-1)
    Plots.plot!(plt, value.(x)[1,:], value.(x)[2,:], value.(x)[3,:])
    return value.(x), value.(v), value.(u)
end

X, V, U = get_feasible_solution!(data, plt)
plt

## SCvx algorithm 

In [ ]:
function project_and_linearize(data, X, V, U)
    z = []
    model = Model(Ipopt.Optimizer);
    set_attribute(model, JuMP.MOI.Silent(), true)

    @variable(model, x[1:3, 1:data.N])
    @variable(model, v[1:3, 1:data.N])
    @variable(model, u[1:3, 1:data.N-1])

    #q
    expressions = []

    for i in 1:data.N-1
        push!(expressions, v[:,i]*data.T/data.N - x[:,i+1] + x[:,i])
    end

    for i in 1:data.N-1
        push!(expressions, u[:,i]*data.T/data.N - data.g*data.T/data.N - v[:,i+1] + v[:,i])
    end

    for k in 1:length(data.R)
        for i in 1:data.N
            push!(expressions, (x[1,i]-data.Px[k])^2 + (x[2,i]-data.Py[k])^2 - data.R[k]^2)
        end
    end

    @objective(model, Min, sum((X-x).^2) + sum((V-v).^2) + sum((U-u).^2))
    for i in 1:(2*(data.N-1) + length(data.R)*data.N)
        con = @constraint(model, expressions[i] <= 0)
        optimize!(model);
        push!(z, (value.(x), value.(v), value.(u)))
        delete(model, con)
    end
    return z
end

function update_solution!(data, z, plt)
    model = Model(Ipopt.Optimizer);
    set_attribute(model, JuMP.MOI.Silent(), true)

    @variable(model, x[1:3, 1:data.N])
    @variable(model, v[1:3, 1:data.N])
    @variable(model, u[1:3, 1:data.N-1])

    @variable(model, g1[1:3, 1:data.N-1])
    @variable(model, g2[1:3, 1:data.N-1])

    #Y
    @constraint(model, x[:,1] .== data.Xo)
    @constraint(model, x[:,end] .== data.Xf)
    @constraint(model, v[:,1] .== data.Vo)
    @constraint(model, v[:,end] .== data.Vf)
    @constraint(model, [i in 1:data.N], sum(v[j,i]^2 for j in 1:3) <= data.Vmax^2)
    @constraint(model, [i in 1:data.N-1], sum(u[j,i]^2 for j in 1:3) <= data.Umax^2)
    @constraint(model, [i in 1:data.N-1], sum(u[:,i].*data.n)^2 >= sum(u[j,i]^2 for j in 1:3) * cos(data.theta)^2)

    #q
    z1 = z[1:data.N-1]
    z2 = z[data.N:2*(data.N-1)]
    z3 = z[2*(data.N-1)+1:end]

    @constraint(model,[i in 1:data.N-1], (v[:,i] - z1[i][2][:,i])*data.T/data.N - (x[:,i+1] - z1[i][1][:,i+1]) + (x[:,i] - z1[i][1][:,i]) .>= 0)
    @constraint(model,[i in 1:data.N-1], (u[:,i] - z2[i][3][:,i])*data.T/data.N - (v[:,i+1] - z2[i][2][:,i+1]) + (v[:,i] - z2[i][2][:,i]) .>= 0)

    for k in 1:length(data.R)
        temp = z3[1+(k-1)*data.N:data.N+(k-1)*data.N]
        @constraint(model, [i in 1:data.N], 2*(temp[i][1][1,i]-data.Px[k])*(x[1,i] - temp[i][1][1,i]) + 
                                            2*(temp[i][1][2,i]-data.Py[k])*(x[2,i] - temp[i][1][2,i]) >= 0)
    end

    @constraint(model, [i in 1:data.N-1], g1[:,i] .>= v[:,i]*data.T/data.N - x[:,i+1] + x[:,i])
    @constraint(model, [i in 1:data.N-1], g1[:,i] .>= - (v[:,i]*data.T/data.N - x[:,i+1] + x[:,i]))
    @constraint(model, [i in 1:data.N-1], g2[:,i] .>= u[:,i]*data.T/data.N - data.g*data.T/data.N - v[:,i+1] + v[:,i])
    @constraint(model, [i in 1:data.N-1], g2[:,i] .>= - (u[:,i]*data.T/data.N - data.g*data.T/data.N - v[:,i+1] + v[:,i]))

    @NLobjective(model, Min, sum(sqrt(sum(u[j,k]^2 for j in 1:3)) for k in 1:data.N-1)^2 + data.lambda*(sum(g1[i,j] for i = 1:3, j = 1:data.N-1) + sum(g2[i,j] for i = 1:3, j = 1:data.N-1)))
    optimize!(model)
    @show sum(sqrt(sum(value(u[j,k])^2 for j in 1:3)) for k in 1:data.N-1)
    Plots.plot!(plt, value.(x)[1,:], value.(x)[2,:], value.(x)[3,:])
    return value.(x), value.(v), value.(u)
end


In [ ]:
z = project_and_linearize(data, X, V, U)
X, Y, Z = update_solution!(data, z, plt)
plt